#Group:6
##Name:
##Anurag Shukla 202318039
## Mahmood Topiwala 202318030
## Vrishmi Parikh 202318013
##Tanaz Pathan 202318056



Imported The Dataset From Kaggle

In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

In [ ]:
!kaggle datasets download -d grouplens/movielens-20m-dataset

movielens-20m-dataset.zip: Skipping, found more recently modified local copy (use --force to force download)


Extracted The Zipped Folder of the Kaggle Dataset

In [ ]:
import zipfile
zip_ref = zipfile.ZipFile('/content/movielens-20m-dataset.zip', 'r')
zip_ref.extractall('/content')
zip_ref.close()

Movies Dataframe

In [ ]:
import pandas as pd
movie_df=pd.read_csv('/content/movie.csv')
movie_df

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy
...,...,...,...
27273,131254,Kein Bund für's Leben (2007),Comedy
27274,131256,"Feuer, Eis & Dosenbier (2002)",Comedy
27275,131258,The Pirates (2014),Adventure
27276,131260,Rentun Ruusu (2001),(no genres listed)


Dropping the unnecessary column for collaborative filtering

In [ ]:
movie_df.drop('genres', axis=1, inplace=True)

Rating Dataframe

In [ ]:
import pandas as pd
rating_df=pd.read_csv('/content/rating.csv')
rating_df

,userId,movieId,rating,timestamp
0,1,2.0,3.5,2005-04-02 23:53:47
1,1,29.0,3.5,2005-04-02 23:31:16
2,1,32.0,3.5,2005-04-02 23:33:39
3,1,47.0,3.5,2005-04-02 23:32:07
4,1,50.0,3.5,2005-04-02 23:29:40
...,...,...,...,...
11000952,76075,1307.0,3.0,2005-10-31 00:28:44
11000953,76075,1343.0,3.0,2005-10-31 00:34:44
11000954,76075,1358.0,3.5,2005-11-06 23:11:45
11000955,76075,1380.0,2.5,2005-09-25 23:05:40


Dropped the Timestamp column

In [ ]:
rating_df.drop('timestamp', axis=1, inplace=True)

Merged Both the Dataframe on Movie_ID

In [ ]:
import pandas as pd

merged_df = pd.merge(rating_df, movie_df, on='movieId')

merged_df


,userId,movieId,rating,title
0,1,2.0,3.5,Jumanji (1995)
1,5,2.0,3.0,Jumanji (1995)
2,13,2.0,3.0,Jumanji (1995)
3,29,2.0,3.0,Jumanji (1995)
4,34,2.0,3.0,Jumanji (1995)
...,...,...,...,...
11000951,75941,105901.0,2.5,Silja - nuorena nukkunut (1956)
11000952,75941,119950.0,3.5,The Magnificent Trio (1966)
11000953,75941,120206.0,2.0,Five Weeks in a Balloon (1962)
11000954,75941,122423.0,4.0,The Love Letter (1998)


Taking Only Authentic User into Considerattion who have given Rating atleast 300 times

In [ ]:
x = merged_df.groupby('userId').count()['rating'] >= 300
authentic_users = x[x].index

In [ ]:
filtered_rating = merged_df[merged_df['userId'].isin(authentic_users)]
filtered_rating

,userId,movieId,rating,title
5,54,2.0,3.0,Jumanji (1995)
7,91,2.0,3.5,Jumanji (1995)
8,116,2.0,2.0,Jumanji (1995)
15,131,2.0,1.0,Jumanji (1995)
16,132,2.0,3.0,Jumanji (1995)
...,...,...,...,...
11000951,75941,105901.0,2.5,Silja - nuorena nukkunut (1956)
11000952,75941,119950.0,3.5,The Magnificent Trio (1966)
11000953,75941,120206.0,2.0,Five Weeks in a Balloon (1962)
11000954,75941,122423.0,4.0,The Love Letter (1998)


Taking Only Famous Movies into Considerattion who have gotten atleast 250 times

In [ ]:
y = merged_df.groupby('movieId').count()['rating']>=250
famous_movies = y[y].index

In [ ]:
final_ratings = filtered_rating[filtered_rating['movieId'].isin(famous_movies)]
final_ratings

,userId,movieId,rating,title
5,54,2.0,3.0,Jumanji (1995)
7,91,2.0,3.5,Jumanji (1995)
8,116,2.0,2.0,Jumanji (1995)
15,131,2.0,1.0,Jumanji (1995)
16,132,2.0,3.0,Jumanji (1995)
...,...,...,...,...
10846195,72541,1572.0,2.0,"Contempt (Mépris, Le) (1963)"
10846196,73258,1572.0,4.0,"Contempt (Mépris, Le) (1963)"
10846197,73382,1572.0,4.5,"Contempt (Mépris, Le) (1963)"
10846199,74115,1572.0,5.0,"Contempt (Mépris, Le) (1963)"


In [ ]:
user_item_matrix = pd.pivot_table(final_ratings, values='rating', index='userId', columns='movieId', fill_value=0)

# Displaying the user-item matrix
user_item_matrix

movieId,1.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0,9.0,10.0,...,112552.0,112556.0,112623.0,112852.0,114180.0,115569.0,115617.0,116797.0,116823.0,118696.0
userId,,,,,,,,,,,,,,,,,,,,,
11,4.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.5,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
24,4.0,0.0,0.0,0.0,2.0,4.0,3.0,0.0,0.0,3.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
54,4.0,3.0,0.0,0.0,3.0,3.0,0.0,0.0,0.0,4.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
58,5.0,0.0,0.0,0.0,0.0,4.5,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
91,4.0,3.5,3.0,0.0,0.0,0.0,2.5,0.0,0.0,4.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76024,4.5,3.5,4.0,0.0,0.0,4.5,0.0,0.0,0.0,4.5,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
76030,3.0,3.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
76049,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
similarity_scores = cosine_similarity(user_item_matrix)
similarity_scores.shape

(8923, 8923)

In [ ]:
import numpy as np
import pandas as pd


def recommend(userId):
    # Check if user_id is in the user_item_matrix index
    if userId not in user_item_matrix.index:
        print(f"User with ID {userId} not found.")
        return None

    # Index fetch
    index = np.where(user_item_matrix.index == userId)[0][0]

    # Get similar items
    similar_items = sorted(list(enumerate(similarity_scores[index])), key=lambda x: x[1], reverse=True)[1:6]

    recommended_movies = set()  # Keep track of recommended movies to avoid duplicates
    data = []

    for i in similar_items:
        temp_df = final_ratings[final_ratings['userId'] == user_item_matrix.index[i[0]]].drop_duplicates('movieId')

        for _, row in temp_df.iterrows():
            movie_id = row['movieId']
            if len(recommended_movies) >= 5:
                break  # Stopping recommending if already recommended 5 movies
            if movie_id not in recommended_movies:
                item = {
                    'Movie ID': movie_id,
                    'Rating': row['rating'],
                    'Movie Name': row['title']
                }
                data.append(item)
                recommended_movies.add(movie_id)

    datadf = pd.DataFrame(data)
    return datadf


In [ ]:
recommend(11)#Recommending for user id 11

,Movie ID,Rating,Movie Name
0,2,2.5,Jumanji (1995)
1,32,3.5,Twelve Monkeys (a.k.a. 12 Monkeys) (1995)
2,47,4.0,Seven (a.k.a. Se7en) (1995)
3,50,4.5,"Usual Suspects, The (1995)"
4,223,4.0,Clerks (1994)


In [ ]:
import numpy as np
import pandas as pd
from surprise import Dataset, Reader, KNNBasic
from surprise.model_selection import train_test_split
from surprise import accuracy



def recommend_and_evaluate(userId):
    # Check if user_id is in the user_item_matrix index
    if userId not in user_item_matrix.index:
        print(f"User with ID {userId} not found.")
        return None

    # Index fetch
    index = np.where(user_item_matrix.index == userId)[0][0]

    # Get similar items
    similar_items = sorted(list(enumerate(similarity_scores[index])), key=lambda x: x[1], reverse=True)[1:6]

    recommended_movies = set()  # Keeping track of recommended movies to avoid duplicates
    data = []

    for i in similar_items:
        temp_df = final_ratings[final_ratings['userId'] == user_item_matrix.index[i[0]]].drop_duplicates('movieId')

        for _, row in temp_df.iterrows():
            movie_id = row['movieId']
            if len(recommended_movies) >= 5:
                break  # Stop recommending if already recommended 5 movies
            if movie_id not in recommended_movies:
                item = {
                    'Movie ID': movie_id,
                    'Rating': row['rating'],
                    'Movie Name': row['title']
                }
                data.append(item)
                recommended_movies.add(movie_id)

    datadf = pd.DataFrame(data)

    # Preparing data for Surprise
    reader = Reader(rating_scale=(1, 5))
    surprise_data = Dataset.load_from_df(final_ratings[['userId', 'movieId', 'rating']], reader)

    # Splitting the data into train and test sets
    trainset, testset = train_test_split(surprise_data, test_size=0.25)

    # Using the User-Based Collaborative Filtering algorithm
    sim_options = {
        'name': 'cosine',
        'user_based': True
    }

    # Creating the model
    model = KNNBasic(sim_options=sim_options)

    # Training the model on the training set
    model.fit(trainset)

    #  predictions on the test set
    predictions = model.test(testset)

    # Evaluate the model using RMSE and MAE
    rmse = accuracy.rmse(predictions)
    mae = accuracy.mae(predictions)

    print(f'RMSE: {rmse}')
    print(f'MAE: {mae}')

    return datadf


In [ ]:
recommend_and_evaluate(11) #Evaluating and Recommending for user id 11

Computing the cosine similarity matrix...
Done computing similarity matrix.
RMSE: 0.9044
MAE:  0.6872
RMSE: 0.9044260705093056
MAE: 0.6872225707489027


,Movie ID,Rating,Movie Name
0,2.0,2.5,Jumanji (1995)
1,32.0,3.5,Twelve Monkeys (a.k.a. 12 Monkeys) (1995)
2,47.0,4.0,Seven (a.k.a. Se7en) (1995)
3,50.0,4.5,"Usual Suspects, The (1995)"
4,223.0,4.0,Clerks (1994)
